In [5]:
from calc import calculate
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

Initial = widgets.Text(value='2000',description='Initial Inv', )
Monthly = widgets.Text(value='400',description='Topup', )
Years = widgets.Text(value='5',description='Years', )
button = widgets.Button(description='Calculate')
out = widgets.Output()

def on_button_clicked(_):
    with out:
        clear_output()
        calculate(Initial.value,Monthly.value,Years.value)
        
button.on_click(on_button_clicked)
widgets.VBox([Initial,Monthly,Years,button,out])